In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import openpyxl
import os

In [57]:
netcdf_files_dir = "D:/bigfiles/netcdf_download/netcdf_files/"
output_dir = "D:/bigfiles/scratch/"
trials_path = output_dir  + 'locations_lat_lon/all_locations_coordinates_test.xlsx'

In [10]:
# -------- Define a function to read the netCDF content  -------- #
def read_netcdf(netcdf_file):
    # Open the NetCDF file
    dataset = nc.Dataset(netcdf_file, 'r')
    # First we list all the variables in the NC file
    print(dataset.variables.keys())
    print("\nVariables and their dimensions:")
    print("Description of each variable:")
    for var in dataset.variables:
        # Fetch the variable object
        variable = dataset.variables[var]
        # Extract description attributes, defaulting to 'N/A' if not present
        long_name = variable.long_name if hasattr(variable, 'long_name') else 'N/A'
        units = variable.units if hasattr(variable, 'units') else 'N/A'
        print(f"{var}:")
        print(f"  Long Name: {long_name}")
        print(f"  Units: {units}\n")
    # Don't forget to close the dataset
    dataset.close()

# define file path to the netCDF file
netcdf_file = os.path.join(netcdf_files_dir, 'pr_2004.nc')
read_netcdf(netcdf_file)

dict_keys(['lon', 'lat', 'day', 'crs', 'precipitation_amount'])

Variables and their dimensions:
Description of each variable:
lon:
  Long Name: longitude
  Units: degrees_east

lat:
  Long Name: latitude
  Units: degrees_north

day:
  Long Name: time
  Units: days since 1900-01-01 00:00:00

crs:
  Long Name: WGS 84
  Units: N/A

precipitation_amount:
  Long Name: pr
  Units: mm



In [32]:
###########################################################################################
# Reading single netcdf file reading and extract variables for different locations        #
# This could be used for variables like pdsi and spei where all the years are in one file #
###########################################################################################


# First define the trials coordinates file path and read it
trials_coords = pd.read_excel(trials_path, sheet_name='Sheet1')
nc_data = nc.Dataset(netcdf_files_dir + 'pdsi.nc', 'r')
start_date = '2001-01-01'
end_date = '2023-12-31'

# Now we have a function to extract the climate variable name from the netcdf file
def auto_climate_variable(nc_data):
    coord_vars = {"lat", "latitude", "lon", "longitude", "time", "day"}

    scored = []

    for name, var in nc_data.variables.items():
        if name.lower() in coord_vars:
            continue

        score = 0
        dims = set(var.dimensions)

        if "lat" in dims or "latitude" in dims:
            score += 1
        if "lon" in dims or "longitude" in dims:
            score += 1
        if "time" in dims or "day" in dims:
            score += 2  # time matters most

        if score > 0:
            scored.append((score, name, var.dimensions))

    if not scored:
        raise ValueError("No climate variable found")

    scored.sort(reverse=True)
    return scored[0][1]

climate_variable_name = auto_climate_variable(nc_data)



# Now extract the climate variable time series for each location
def extract_netcdf_variables(netcdf_file, locations_df,climate_variable_name, start_date, end_date, output_path):
    
    for index, row in locations_df.iterrows():
        location = row['Location']
        latitude = row['Latitude']
        longitude = row['Longitude']
        # Retrieve the latitude and longitude data
        lat = netcdf_file.variables['lat'][:]
        lon = netcdf_file.variables['lon'][:]
        
        # Your location's latitude and longitude
        location_lat =  latitude # Replace with the specific latitude
        location_lon = longitude  # Replace with the specific longitude

        # Find the nearest index for the specified location
        lat_idx = np.abs(lat - location_lat).argmin()
        lon_idx = np.abs(lon - location_lon).argmin()

        # Retrieve climate variable for all times for the specified location
        climate_variable = netcdf_file.variables[climate_variable_name][:, lat_idx, lon_idx]  # Assuming time is the first dimension

        # Convert to a DataFrame
        locations_df = pd.DataFrame({
            'data': climate_variable
        })

        if climate_variable_name == 'air_temperature':
        #Temperature in gridMET is in Kelvin. Convert it to celsius. If you don't have temperature netclocations_df file, then comment the line below
            locations_df = locations_df - 273.15

        
        lat_num = lat[lat_idx]
        lon_num = lon[lon_idx]
        locations_df.columns = ['data']

        # Assuming the reference date is January 1, 1900
        reference_date = datetime(1900, 1, 1)

        days = netcdf_file.variables['day'][:]
        # Convert days array to pandas Series
        days_series = pd.Series(days)

        # Convert to Gregorian dates using vectorized operations
        gregorian_dates_pd = reference_date + pd.to_timedelta(days_series, unit='D')

        days = gregorian_dates_pd

        locations_df['dates'] = days
        df_new = locations_df[['dates','data']]
        df_new.columns = ['date', climate_variable_name]
        # filter date form start_year to end_year
        df_new = df_new[(df_new['date'] >= start_date) & (df_new['date'] <= end_date)]
        output_file = os.path.join(output_path, f'{location}.xlsx')
        df_new.to_excel(output_file,index = False)


extract_netcdf_variables(nc_data, trials_coords, climate_variable_name,
                         start_date, end_date, output_dir)

In [ ]:
#########----- write a code to extract multiple netcdf files for multiple locations and multiple years ---------#####
#####################################################################################################################

def _pick_time_var(ds):
    """Return the name of the time variable (common options: 'time', 'day')."""
    for cand in ["time", "day", "days"]:
        if cand in ds.variables:
            return cand
    # fallback: look for a 1D variable with 'since' in units
    for vname, v in ds.variables.items():
        if getattr(v, "ndim", 0) == 1 and isinstance(getattr(v, "units", ""), str) and "since" in v.units.lower():
            return vname
    raise KeyError("Could not find a time variable (expected 'time' or 'day', or units like 'days since ...').")

def _pick_data_varnames(ds):
    """Return likely climate variable names excluding coordinate vars."""
    exclude = {"lat", "latitude", "lon", "longitude", "time", "day", "days", "crs"}
    return [v for v in ds.variables.keys() if v.lower() not in exclude]

def _to_datetime_index(time_vals, time_units, time_calendar):
    """Convert numeric time to pandas datetime."""
    dt = nc.num2date(time_vals, units=time_units, calendar=time_calendar)
    # num2date may return cftime objects; pandas handles them via to_datetime on string conversion
    return pd.to_datetime([str(x) for x in dt])

def _normalize_lon_for_dataset(target_lon, lon_array):
    """
    Normalize target_lon to match dataset lon convention.
    If dataset lon is 0..360, convert negative lon to 0..360.
    If dataset lon is -180..180, keep as-is.
    """
    lon_min, lon_max = float(np.nanmin(lon_array)), float(np.nanmax(lon_array))
    if lon_min >= 0 and lon_max > 180:
        # dataset is likely 0..360
        return target_lon % 360
    return target_lon


def extract_netcdf_variables(
    locations_df,
    year_list,
    variable_list,
    netcdf_dir,
    out_dir,
    start_date=None,
    end_date=None,
):
    start_dt = pd.to_datetime(start_date) if start_date else None
    end_dt   = pd.to_datetime(end_date) if end_date else None

    os.makedirs(out_dir, exist_ok=True)

    for _, row in locations_df.iterrows():
        location  = str(row["Location"])
        latitude  = float(row["Latitude"])
        longitude = float(row["Longitude"])

        all_parts = []  # accumulate (variable, year) chunks

        for vcode in variable_list:
            for year in year_list:
                netcdf_file = os.path.join(netcdf_dir, f"{vcode}_{year}.nc")
                if not os.path.exists(netcdf_file):
                    print(f"⚠️ Missing file, skipping: {netcdf_file}")
                    continue

                with nc.Dataset(netcdf_file, "r") as ds:
                    # coords
                    lat_name = "lat" if "lat" in ds.variables else ("latitude" if "latitude" in ds.variables else None)
                    lon_name = "lon" if "lon" in ds.variables else ("longitude" if "longitude" in ds.variables else None)
                    if lat_name is None or lon_name is None:
                        raise KeyError(f"{netcdf_file}: Could not find lat/lon variables.")

                    lat = ds.variables[lat_name][:]
                    lon = ds.variables[lon_name][:]

                    adj_lon = _normalize_lon_for_dataset(longitude, lon)

                    lat_idx = int(np.abs(lat - latitude).argmin())
                    lon_idx = int(np.abs(lon - adj_lon).argmin())

                    # time
                    tname = _pick_time_var(ds)
                    tvar = ds.variables[tname]
                    time_vals = tvar[:]
                    time_units = getattr(tvar, "units", "days since 1900-01-01")
                    time_calendar = getattr(tvar, "calendar", "standard")
                    dates = _to_datetime_index(time_vals, time_units, time_calendar)

                    # data variables inside file
                    data_vars = _pick_data_varnames(ds)
                    if len(data_vars) == 0:
                        print(f"⚠️ No data variables found in {netcdf_file}, skipping.")
                        continue

                    df = pd.DataFrame({"DATE": dates})

                    for varname in data_vars:
                        var = ds.variables[varname]
                        dims = getattr(var, "dimensions", ())
                        ndims = len(dims)

                        # Expecting (time, lat, lon) or (time, y, x)
                        if ndims >= 3:
                            values = var[:, lat_idx, lon_idx]
                        elif ndims == 1 and dims and dims[0].lower() in {"time", "day", "days"}:
                            values = var[:]
                        else:
                            print(f"  ⚠️ Skipping {varname} (unsupported dims={dims})")
                            continue

                        units = str(getattr(var, "units", "")).lower()

                        # Kelvin -> Celsius if units suggest Kelvin
                        if units in {"k", "kelvin"} or "kelvin" in units:
                            values = values - 273.15

                        # ✅ Column name exactly from the nc variable
                        col = varname

                        # If there is a collision (same varname appears from different files), make it unique
                        if col in df.columns:
                            col = f"{varname}_{vcode}"

                        # If multiple data_vars exist, keep them distinct
                        if col in df.columns and varname != col:
                            col = f"{col}_{varname}"

                        df[col] = values

                    # date filter
                    if start_dt is not None:
                        df = df[df["DATE"] >= start_dt]
                    if end_dt is not None:
                        df = df[df["DATE"] <= end_dt]

                    df["Location"] = location
                    df["YearFile"] = year
                    #df["SourceVar"] = vcode
                    all_parts.append(df)

                print(f"✅ Extracted {location} | {vcode}_{year}.nc (nearest grid: lat[{lat_idx}], lon[{lon_idx}])")

        if len(all_parts) == 0:
            print(f"⚠️ Nothing extracted for {location}.")
            continue

        out = pd.concat(all_parts, ignore_index=True)

        # make sure DATE is datetime
        out["DATE"] = pd.to_datetime(out["DATE"])

        # Pivot to one row per Location+DATE
        wide = (out
                .pivot_table(index=["Location", "DATE"],
                            #values=["TMIN_C", "TMAX_C"],
                            aggfunc="first")   # first non-null
                .reset_index()
                .sort_values(["Location", "DATE"])
        )

        save_path = os.path.join(out_dir, f"{location}_daily.xlsx")
        wide.to_excel(save_path, index=False)

In [62]:

years = list(range(1979, 2024))  
variable_list = ["tmmn", "pr"] # test with two variables
#variable_list = ['tmmx', 'tmmn', 'vpd', 'pet', 'aet', 'pdsi', 'spei', 'sm', 'ws', 'srad', 'rhmax', 'rhmin', 'sph']
extract_netcdf_variables(
    pd.read_excel(trials_path, sheet_name='Sheet1'),
    years,
    variable_list,
    netcdf_files_dir,
    out_dir=output_dir,
    start_date="2001-01-01",
    end_date="2023-12-31",
)

✅ Extracted Almira | tmmn_1979.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1980.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1981.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1982.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1983.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1984.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1985.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1986.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1987.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1988.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1989.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1990.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1991.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1992.nc (nearest grid: lat[37], lon[141])
✅ Extracted Almira | tmmn_1993.nc (nearest grid: